# Test hypotézy #
Prvně je třeba nahrát potřebné knihovny:

In [84]:
import pandas as pd
import numpy as np
import scipy.stats as stats

## Hypotéza 1: ##
*"Na silnicích první třídy se při nehodách umíralo se stejnou pravděpodobností jako na
silnicích třetí třídy."*

K ověření této hypotézy využijeme testu dobré shody χ², mezivýpočty budeme průběžně zobrazovat.

V prvé řadě uložíme do datového rámce pouze ty nehody, při kterých došlo k úmrtí:

In [85]:
df1=pd.read_pickle("accidents.pkl.gz").dropna(subset="p13a")
df1=df1[(df1["p13a"]>0)]

Z těchto dat je třeba ještě získat počet nehod s úmrtím na silnicích 1. třídy a nehod s umrtím na silnicích 3. třídy.

In [86]:
first_class_accidents=len(df1[df1['p36'] == 1])
third_class_accidents=len(df1[df1['p36'] == 3])
print("Celkový počet smrtelných nehod na silnicích:")
print("\nprvní třídy: "+str(first_class_accidents))
print("\ndruhé třídy: "+str(third_class_accidents))

Celkový počet smrtelných nehod na silnicích:

první třídy: 1104

druhé třídy: 536


Nyní stanovíme očekávaný počet nehod, protože se hypotéza hovoří o stejné pravděpodobnosti, tento počet se zjistí z podílu součtu smrtelných nehod na silnicích 1. a 3. třídy:

In [87]:
expected_accidents=(first_class_accidents+third_class_accidents)/2
print("očekávaný počet nehod: "+str(expected_accidents))

očekávaný počet nehod: 820.0


Po zjištění očekávané hodnoty vytvoříme kontigenční tabulku:

In [88]:
contingency_table=np.array([
    [first_class_accidents,third_class_accidents],
    [expected_accidents,expected_accidents]
])

Nic nám již nebrání v výpočtu statistiky testu dobré shody χ² a jeho p-hodnoty:

In [89]:
chi,p_value=stats.chisquare(f_obs=contingency_table[0], f_exp=contingency_table[1])
print("statistika χ² je: "+str(chi))
print("p-hodnota je:     "+str(p_value))

statistika χ² je: 196.7219512195122
p-hodnota je:     1.0844372520926372e-44


Nakonec zjistíme pravdivost hypotézy. Pokud je statistika χ² nulová nebo pokud je p-hodnota větší jak 0.05 (pro 95% jistotu), není mezi nehodovostí statisticky významný rozdíl je hypotéza je tedy pravdivá, v pokud je statistika nenulová a p-hodnota je menší než 0.05, hypotéza je nepravdivá. Následující kód rozhodne o pravdivosti hypotézy:

In [90]:
if chi == 0:
    print("Mezi nehodami obou tříd silnic je NULOVÝ rozdíl,")
    print("hypotéza je PRAVDIVÁ")
else:
    is_hypothesis_correct=p_value>0.05
    if(is_hypothesis_correct):
        print("Mezi nehodami obou tříd silnic je statisticky NEVÝZNAMNÝ rozdí,l")
        print("hypotéza je PRAVDIVÁ")
    else:
        print("Mezi nehodami obou tříd silnic je statisticky VÝZNAMNÝ rozdíl,")
        print("hypotéza je NEPRAVDIVÁ")

Mezi nehodami obou tříd silnic je statisticky VÝZNAMNÝ rozdíl,
hypotéza je NEPRAVDIVÁ


## Hypotéza 2: ##
*"Při nehodách vozidel značky Škoda je škoda na vozidle nižší než při nehodách vozidel Audi."*

Pro ověření této hypotézy je třeba určit vhodný statistiký test. Rozdělení škod nemusí být normální, proto použiji Mann-Whitney U test.

Opět prvně získáme konkrétní data:

In [91]:
df2=pd.read_pickle("accidents.pkl.gz").dropna(subset="p45a")
df_skoda=df2[df2["p45a"] == 39]
df_audi=df2[df2["p45a"] == 2]

Nyní získáme u-statistiku a p-hodnotu pro zjištění, zdali je mezi vzorky statisticky významný rozdíl:

In [92]:
u_statistic, p_value = stats.mannwhitneyu(df_skoda["p53"], df_audi["p53"], alternative='two-sided')

print("U-statistika: "+str(u_statistic))
print("P-hodnota:    "+str(p_value))

U-statistika: 871624883.5
P-hodnota:    4.553996536432389e-220


Ze získaných hodnot rozhodneme, zdali je při nenulové u-hodnotě na základě p-statistiky a jistoty zdali je mezi vzorky statisticky významný rozdíl:

In [93]:
is_significant=p_value<0.05

Pokud je statisticky rozdíl významný, je třeba rozhodnout která značka měla při nehodách větší škody, například pomocí mediánu, dále rozhodneme o pravdivosti hypotézy.

In [94]:
if is_significant:
    print("Rozdíl JE významný")
    if df_audi["p53"].median()>df_skoda["p53"].median():
        print("Na automobilech AUDI byly způsobeny větší škody")
        print("Hypotéza je PRAVDIVÁ")
    elif df_audi["p53"].median()<df_skoda["p53"].median():
        print("Na automobilech ŠKODA byly způsobeny větší škody")
        print("Hypotéza je NEPRAVDIVÁ")
else:
    print("Rozdíl NENÍ významný")


Rozdíl JE významný
Na automobilech AUDI byly způsobeny větší škody
Hypotéza je PRAVDIVÁ
